In [149]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,accuracy_score,confusion_matrix
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.manifold import MDS
from sklearn.preprocessing import Imputer,StandardScaler,MinMaxScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline
import seaborn as sns
from __future__ import division
%matplotlib inline

In [150]:
X = pd.read_csv('S&P500_2017.csv')
X = X.drop(['Date'],axis=1)

In [174]:
X.head()

A        AAL         AAP        AAPL       ABBV        ABC  \
0  46.074703  45.816051  170.237915  113.847588  59.784031  80.855621   
1  46.679256  46.211868  171.634933  113.720161  60.626999  82.862099   
2  46.124260  45.410336  171.515182  114.298462  61.086800  81.902901   
3  47.561314  45.726990  169.269974  115.572708  61.105961  82.999107   
4  47.709969  46.587898  169.170197  116.631294  61.508286  83.664680   

         ABT         ACN        ADBE        ADI    ...             XL  \
0  37.982937  114.158501  103.480003  70.577911    ...      36.900692   
1  38.284462  114.432961  104.139999  70.431908    ...      37.027840   
2  38.615170  112.717552  105.910004  69.419624    ...      36.607288   
3  39.665661  114.001671  108.300003  69.692154    ...      36.568165   
4  39.626755  112.727356  108.570000  70.023094    ...      35.952015   

        XLNX        XOM       XRAY        XRX        XYL        YUM  \
0  57.513260  86.670731  58.296070  26.833357  48.875042  61.848686   
1  57.094593  85.717155  58.773418  27.845938  49.603493  62.073734   
2  56.403305  84.439362  58.007671  27.729099  49.150673  62.279217   
3  57.493786  84.391678  57.967899  27.300703  48.806133  63.032623   
4  57.503525  82.999451  58.146900  27.144920  48.599411  63.208748   

          ZBH       ZION        ZTS  
0  102.513084  42.586197  53.118965  
1  103.455551  43.197666  53.634392  
2  104.120262  42.497429  53.455975  
3  104.130180  42.773582  53.624477  
4  106.154053  42.310051  53.475800  

[5 rows x 503 columns]

In [197]:
r = 0.035/365
n = 1000

In [198]:
Times = [(t1,t1+np.random.choice(range(1,30))) for t1 in [np.random.choice(range(220))for _ in range(n)] ]

In [199]:
M=X.fillna(X.mean())
M = M.pct_change()
Z = pd.DataFrame(columns=M.columns)
Z = Z.T
Z['Mean'] = M.mean()
Z['Std'] = M.std()
Z['Skew'] = M.skew()
Z['Kurt'] = M.kurt()

$$K=S_0e^{rt}$$

In [200]:
Y = pd.DataFrame(columns=['Mean','Std','Skew','Kurt','I','F','D'],index=range(n))
for i in range(n):
    ran = np.random.choice(503)
    t = (Times[i][1]-Times[i][0])/365
    S0 = X.T.iloc[ran][Times[i][0]]
    K = S0*np.e**(r*t)
    St =  X.T.iloc[ran][Times[i][1]] 
    d=  (K <=St)*1
    Y.iloc[i]=list(Z.iloc[ran])+[Times[i][0],Times[i][1],d]

In [201]:
Y.head()

Mean         Std      Skew     Kurt    I    F  D
0 -0.000283331    0.016763  -1.94298  13.3698   26   51  0
1  0.000370343  0.00892925  0.538415   3.0018  164  192  0
2   0.00178651   0.0148502   1.59569  11.0888   59   71  0
3  0.000424116   0.0161442  -4.13594  38.4415  122  134  1
4   0.00027734   0.0119757  -1.03356   6.0076   17   22  1

In [202]:
for col in Y.columns:
    Y[col] = Y[col].apply(pd.to_numeric, errors='coerce')

In [203]:
W = Y[['Mean','Std','Skew','Kurt','I','F']]
y = Y[['D']]

In [204]:
Xt,Xv,yt,yv =train_test_split(W,y,train_size=0.7)
for d in [Xt,Xv,yt,yv]:
    print d.shape

(700, 6)
(300, 6)
(700, 1)
(300, 1)


In [205]:
def metricas(model,Xt,Xv,yt,yv):
    print roc_auc_score(y_score=model.predict_proba(Xt)[:,1],y_true=yt)
    print roc_auc_score(y_score=model.predict_proba(Xv)[:,1],y_true=yv)
    print accuracy_score(y_pred=model.predict(Xt),y_true=yt)
    print accuracy_score(y_pred=model.predict(Xv),y_true=yv)
    print confusion_matrix(y_pred=model.predict(Xt),y_true=yt)
    print confusion_matrix(y_pred=model.predict(Xv),y_true=yv)

In [206]:
model =LogisticRegression()
model.fit(Xt,yt)
metricas(model,Xt,Xv,yt,yv)

0.584091925319
0.542669683258
0.585714285714
0.563333333333
[[ 17 270]
 [ 20 393]]
[[  7 123]
 [  8 162]]


In [207]:
from sklearn.tree import DecisionTreeClassifier
param_grid = dict(criterion=['gini','entropy'],
                 max_depth=range(1,11),
                 min_samples_split = range(100,210,10)
                 )
model = DecisionTreeClassifier()
grid = GridSearchCV(cv=10,n_jobs=-1,scoring='roc_auc',
                    estimator=model,
                    param_grid=param_grid)


grid.fit(W,y)

print grid.best_estimator_
print grid.best_params_
print grid.best_score_

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=5,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=100,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
{'min_samples_split': 100, 'criterion': 'entropy', 'max_depth': 5}
0.626796352392


In [208]:
model = grid.best_estimator_
metricas(model,Xt,Xv,yt,yv)

0.69581797167
0.728687782805
0.645714285714
0.683333333333
[[161 126]
 [122 291]]
[[ 81  49]
 [ 46 124]]


In [209]:
from xgboost.sklearn import XGBClassifier

In [218]:
xgb = XGBClassifier()
xgb.fit(W,y)
metricas(xgb,Xt,Xv,yt,yv)

0.862774295332
0.884389140271
0.777142857143
0.796666666667
[[182 105]
 [ 51 362]]
[[ 87  43]
 [ 18 152]]


In [225]:
from WeightofEvidence import WoE

In [212]:
woe = WoE([],['Mean','Std','Skew','Kurt','I','F'])

In [221]:
woe.fit(W,y,bins=range(2,30))

In [223]:
woe.IV

Mean       Std      Skew      Kurt         I         F
0  0.000016  0.000069  0.061809  0.033441  0.265813  0.237345

In [224]:
Ww = woe.transform(W)